In [ ]:
run_component_tests = False

# Environment

In [ ]:
from BaseClasses import triggering, eFMU, optimization_root

if run_component_tests:
    import os
    import pandas as pd
    import numpy as np
    import matplotlib.pyplot as plt
    %matplotlib inline
    
    import sys
    path_forecasting = r'D:\Users\emma\Documents\GitHub\SmartInverter\smartinverter_optimization'
    sys.path.append(os.path.join(path_forecasting, 'SlowActing'))

# ForecastMPC

In [ ]:
from ComputeTariff import tariff_pge_e19
import copy
import time
import warnings

class mpc_input(eFMU):   
    def __init__(self):
        self.input = {
            'time':None,
            'p_pv':None,
            'p_pv_hist':None,
            'p_pv_hist_init':None,
            'config':None,
            'models':None,
            'external_weather_forecast':False,
            'location_latitude':None,
            'location_longitude':None,
            'tz_st':None,
            'wf_prev':None,
            'forecast_pv_prev':None,
            'forecast_load_prev':None
        }        
        self.output = {
            'generation_pv':None,
            'load_demand':None,
            'p_pv_hist_update':None,
            'weather_forecast':None,
            'forecast_pv_sarima':None,
            'forecast_pv_nn':None,
            'forecast_pv_next':None,
            'forecast_pv_diff':None,
            'forecast_pv_error':None,
            'forecast_load_next':None,
            'forecast_load_diff':None,
            'forecast_load_error':None
        } 
        self.pvforecast = None
        from DefaultConfiguration import get_input_example2, get_input_example3
        #self.staticinput = get_input_example2() # Flexlab load
        self.staticinput = get_input_example3() # B90 load
        self.init = True
        
    def emulate(self, now):
        df = self.staticinput.copy(deep=True)
        df.loc[df.index[-1]+pd.DateOffset(hours=1)] = df.iloc[0]
        df = df.resample('1T').asfreq()
        for c in df.columns:
            if c in ['load_demand','generation_pv']:
                df[c] = df[c].interpolate()
            else:
                df[c] = df[c].ffill()
        ix_start = pd.to_datetime(now.strftime('%Y-%m-%d %H:%M:00'))
        ix_df = min(df.index[(df.index.hour==ix_start.hour) & (df.index.minute==ix_start.minute)])
        df = df.loc[ix_df::].append(df.loc[df.index[0]:ix_df])
        df = df[~df.index.duplicated(keep='first')] 
        df = df.reset_index(drop=True)
        df.index = [ix_start + pd.DateOffset(minutes=ix) for ix in df.index]
        df['generation_pv'] = df['generation_pv'] * 1e3
        df['load_demand'] = df['load_demand'] * 1e3
        return df
    
    def filter_pvhist(self, pv_hist_raw):
        pv_hist = pv_hist_raw.copy(deep=True).resample('1T').interpolate().resample('15T',base=pv_hist_raw.index[-1].minute).asfreq()
        pv_hist = pv_hist.mask((pv_hist.index.hour<5) | (pv_hist.index.hour>20), 0) # Filter night time
        pv_hist = pv_hist.mask(pv_hist<10, 0) # Filter negativ PV
        pv_hist = pv_hist.mask(pv_hist.diff(1).abs()>500, np.nan) # Filter slope
        #pv_hist = pv_hist.dropna()
        pv_hist = pv_hist.interpolate()
        return pv_hist
    
    def compute(self):
        error = ''
        now = (pd.to_datetime(self.input['time'], unit='s').replace(microsecond=0, nanosecond=0)+\
               pd.DateOffset(hours=self.input['tz_st'])).to_pydatetime()
        #print 'NOW', datetime.now(), now
        #print 'PV_now', self.input['p_pv']
        ts = 5 #min Optimization and histroic data
        ts_fc = 15 #min Forecast
        horizon = 23 #hours
        # Initialization
        #print self.input['p_pv_hist']
        if self.init and self.input['p_pv_hist_init']:
            error += 'Assign PV History (None)\n'
            self.input['p_pv_hist'] = self.input['p_pv_hist_init']
        #print len(self.input['p_pv_hist']), (60/ts_fc*horizon*2)
        if not self.input['p_pv_hist']:
            error += 'Initialize PV History (None)\n'
            ix = [now.replace(second=0, microsecond=0)-timedelta(minutes=t*ts_fc+ts_fc) for t in range(60/ts_fc*horizon*2)]
            self.input['p_pv_hist'] = pd.Series([0.0]*(60/ts_fc*horizon*2), index=pd.to_datetime(ix))
            self.input['p_pv_hist'].index = (self.input['p_pv_hist'].index.astype(np.int64) / 10 ** 6).astype(np.str)
            self.input['p_pv_hist'] = self.input['p_pv_hist'].to_dict()
            warnings.warn('Initialize PV History (None)', Warning)
        # Read PV hist_raw
        pv_hist_raw = pd.Series(self.input['p_pv_hist'])
        pv_hist_raw.index = pd.to_datetime(pv_hist_raw.index, unit='ms')
        # Add current measurement
        pv_hist_raw.loc[now.replace(second=0, microsecond=0)] = float(self.input['p_pv'])
        #print pv_hist_raw.iloc[[0,1,2,3,4,5,-5,-4,-3,-2,-1]]
        # Render pv_hist (for current timestep)
        pv_hist = self.filter_pvhist(pv_hist_raw)
        #pv_hist.plot()
        #print pv_hist.iloc[[0,1,2,3,4,5,-5,-4,-3,-2,-1]]
        #print len(pv_hist), (60/ts_fc*horizon*2)
        
        if not len(pv_hist) > (60/ts_fc*horizon):
            #print len(self.input['p_pv_hist']), (60/ts_fc*horizon*2)
            error += 'Initialize PV History (len<len)\n'
            ix = [now.replace(second=0, microsecond=0)-timedelta(minutes=t*ts_fc+ts_fc) for t in range(60/ts_fc*horizon*2)]
            self.input['p_pv_hist'] = pd.Series([0.0]*(60/ts_fc*horizon*2), index=pd.to_datetime(ix))
            #self.input['p_pv_hist'].index = (self.input['p_pv_hist'].index.astype(np.int64) / 10 ** 6).astype(np.str)
            #self.input['p_pv_hist'] = self.input['p_pv_hist'].to_dict()
            warnings.warn('Initialize PV History (len<len)', Warning)
        #print 'Error', error
        # Get inputs and update historian
        #pv_hist = pd.Series(self.input['p_pv_hist'])
        #pv_hist.index = pd.to_datetime(pv_hist.index, unit='ms')
        #print pv_hist[np.isnan(pv_hist)]
        #pv_hist.loc[now.replace(second=0, microsecond=0)] = float(self.input['p_pv'])
        #print pv_hist.iloc[[0,1,2,3,4,5,-5,-4,-3,-2,-1]]
        #print pv_hist.iloc[[0,1,2,3,4,5,-5,-4,-3,-2,-1]]
        #pv_hist = pv_hist.iloc[-(60/ts*horizon*2)::]
        #pv_hist = pv_hist.resample('15T', base=pv_hist.index[-1].minute).mean()
        #print 'A', len(pv_hist)
        #print pv_hist
        #print pv_hist[-5::]
        #print pv_hist.resample('1T').interpolate()[-20::]
        #pv_hist = pv_hist.resample('1T').interpolate().resample('15T',base=pv_hist.index[-1].minute).asfreq() # Interpolate
        #print pv_hist[-5::]
        #print 'B', len(pv_hist)
        #pv_hist = pv_hist.dropna()
        #print 'C', len(pv_hist)
        #print throwanerror
        #print pv_hist.iloc[[0,1,2,3,4,5,-5,-4,-3,-2,-1]]
        # Get PV forecast
        if not self.pvforecast:
            from MPCforecasting_V2 import Forecasting, get_forecast_noaa
            models = copy.deepcopy(self.input['models'])
            self.pvforecast = Forecasting(models=models, config=self.input['config'])
            self.get_forecast = get_forecast_noaa
            #print self.pvforecast.loaded_models
        # Get weather forecast
        #print self.input['external_weather_forecast']
        if not isinstance(self.input['external_weather_forecast'], pd.Series):
            if self.input['location_longitude'] > 0: print 'Warning: Longitufe is typically > 0!'
            config = {}
            config['location_latitude'] = self.input['location_latitude']
            config['location_longitude'] = self.input['location_longitude']
            config['tz_st'] = self.input['tz_st']
            wf_prev = pd.Series(self.input['wf_prev'])
            
            # This section is just for testing purposes
            timeout = 5
            if 'test_forcast' in self.input.keys():
                timeout = 0.001
            # End testing purpose
            
            wf, tmp = self.get_forecast(now, config, wf_prev=wf_prev, timeout=timeout)
            error += tmp
            wf['timeIndex'] = pd.to_datetime(wf['timeIndex'])
        else:
            print 'ERROR!, Not implemented!'
        #print wf
        #print 'WARNING: Temporarly shift pv_hist by 1 hour (to PDT)!'
        #pv_hist.index = pv_hist.index.shift(4)
        #print pv_hist.iloc[[0,1,2,3,4,5,-5,-4,-3,-2,-1]]
        #print pv_hist.index        
        pv_fc = self.pvforecast.do_forecast(pv_hist, wf)
        #print pv_fc.iloc[[0,1,2,3,4,5,-5,-4,-3,-2,-1]]
        #pv_hist.plot()
        #pv_fc.plot()
        #print pv_fc
        df = pd.DataFrame({}, index=pv_fc.index)
        df.index = pd.to_datetime(df.index)
        df = df.loc[df.index[0]:(df.index[0]+pd.DateOffset(hours=24))]
        df['generation_pv'] = pv_fc
        #print df
        df = df.resample('15T',base=df.index[0].minute).asfreq()
        #print 'WARNING: Temporarly shift pv_fc by -1 hour! (to PST)'
        #df.index = df.index.shift(-4)
        #print df
        df.loc[now.replace(second=0, microsecond=0), 'generation_pv'] = float(self.input['p_pv'])
        df = df.sort_index()
        df = df.resample('15T',base=df.index[0].minute).asfreq()
        #print df
        #print df.iloc[[0,1,2,3,4,5,-5,-4,-3,-2,-1]]
        #df['generation_pv'].plot()
        #plt.legend()
        #plt.show()

        #print df
        #df = self.compute_periods(df, self.input['tariff'], tz_df=self.input['tz_st'], tz_local=self.input['tz_local'])
        
        emu = self.emulate(now)
        df['load_demand'] = emu['load_demand'] # FIMXE
        #df['tariff_regdn'] = emu['tariff_regdn'] # FIMXE
        #df['tariff_regup'] = emu['tariff_regup'] # FIMXE
        
        # Scale PV and building load
        df['generation_pv'] = df['generation_pv']/1e3
        df['load_demand'] = df['load_demand']/1e3
        if 'scale_pv' in self.input['config'].keys():
             df['generation_pv'] = df['generation_pv'] * self.input['config']['scale_pv']
        if 'scale_load' in self.input['config'].keys():
             df['load_demand'] = df['load_demand'] * self.input['config']['scale_load']       
       
        # Output
        self.output['forecast_pv_next'] = float(df['generation_pv'].values[1]*1e3)
        self.output['forecast_load_next'] = float(df['load_demand'].values[1]*1e3)
        df.index = (df.index.astype(np.int64) / 10 ** 6).astype(np.str)
        df = df.round(2).to_dict()
        for k,v in df.iteritems():
            self.output[k] = v
        #print self.output['generation_pv']
        #pv_fc.index = (pv_fc.index.astype(np.int64) / 10 ** 6).astype(np.str)
        #self.output['p_pv_forecast'] = pv_fc.round(10).to_dict()
        #print pv_hist
        #print len(pv_hist)
        pv_hist_raw = pv_hist_raw.loc[pv_hist_raw.index[-1]-pd.DateOffset(days=2)::]
        #print pv_hist_raw.iloc[[0,1,2,3,4,5,-5,-4,-3,-2,-1]]
        
        #print len(pv_hist)
        pv_hist_raw.index = (pv_hist_raw.index.astype(np.int64) / 10 ** 6).astype(np.str)
        self.output['p_pv_hist_update'] = pv_hist_raw.round(2).to_dict()
        
        
        #print len(self.output['p_pv_hist_update']), (60/ts_fc*horizon*2)
        
        wf['timeIndex'] = time.mktime(wf['timeIndex'].timetuple())
        self.output['weather_forecast'] = wf.to_dict()
        forecast_pv_sarima = self.pvforecast.forecast['sarima']
        forecast_pv_sarima.index = (forecast_pv_sarima.index.astype(np.int64) / 10 ** 6).astype(np.str)
        self.output['forecast_pv_sarima'] = forecast_pv_sarima.to_dict()
        forecast_pv_nn = self.pvforecast.forecast['nn']
        forecast_pv_nn.index = (forecast_pv_nn.index.astype(np.int64) / 10 ** 6).astype(np.str)
        self.output['forecast_pv_nn'] = forecast_pv_nn.to_dict()
        
        # forecast statistics
        if not self.init:
            self.output['forecast_pv_diff'] = float(self.input['forecast_pv_prev']) - float(self.input['p_pv'])
            if self.input['p_pv'] <> 0:
                self.output['forecast_pv_error'] = self.output['forecast_pv_diff'] / float(self.input['p_pv'])
            else:
                self.output['forecast_pv_error'] = -1
            self.output['forecast_load_diff'] = -1
            self.output['forecast_load_error'] = -1
    
        #print self.output
        
        self.init = False
        error = error if len(error)>0 else 'ok'
        return error
        
import pytz
from SMAPDownload_V1 import API_FL
from flexgrid_dictionary_V1 import get_felxgrid_smap

def generate_smap_dict(smap_channels, smap_dict):
    channels = {}
    for c in smap_channels:
        channels[smap_dict[c]['uuid']] = c
    return channels  

import pandas as pd
import numpy as np
from datetime import datetime, timedelta

def get_PV_smap(now, readings, inv_id, tz=-8):
    fmt = '%Y-%m-%dT%H:%M:%S'
    smap_channels = ['DC_P_W','Batt_P_W']
    smap_dict = readings[inv_id]['channels']
    dst = datetime.now(pytz.timezone('US/Pacific')).timetuple().tm_isdst
    now = (pd.to_datetime(now, unit='s').replace(microsecond=0, nanosecond=0) + 
           pd.DateOffset(hours=tz)+pd.DateOffset(hours=dst)) # Convert to local time; SMAP returns standard time
    smap = API_FL(','.join(generate_smap_dict(smap_channels, smap_dict)),
                  (now-timedelta(days=3)).strftime(fmt), now.to_pydatetime().strftime(fmt),
                  'flexstorevh.lbl.gov')
    smap = smap.rename(columns=generate_smap_dict(smap_channels, smap_dict))
    smap['DC_P_W'] = smap['DC_P_W'].mask((smap['DC_P_W']>8600) | (smap['DC_P_W']<-8600) , np.nan)
    smap['DC_PV_W'] = smap['DC_P_W'] + smap['Batt_P_W']
    smap['DC_PV_W'] = smap['DC_PV_W'].mask((smap.index.hour<5) | (smap.index.hour>20), 0) # Filter night time
    smap['DC_PV_W'] = smap['DC_PV_W'].mask(smap['DC_PV_W']<10, 0) # Filter negativ PV
    smap['DC_PV_W'] = smap['DC_PV_W'].mask(smap['DC_PV_W'].diff(1).abs()>500, np.nan) # Filter slope
    return smap
    
def warmstart_mpc_input(now, inv_id=1, tz=-8):
    readings = get_felxgrid_smap()
    if inv_id == 'sum':
        p_pv_hist = pd.DataFrame()
        for inv_id in [1,2,3]:
            smap = get_PV_smap(now, readings, inv_id, tz=tz).resample('1T').mean()
            if p_pv_hist.empty:
                p_pv_hist = smap['DC_PV_W']
            else:
                p_pv_hist += smap['DC_PV_W']
    else:
        smap = get_PV_smap(now, readings, inv_id, tz=tz)
        p_pv_hist = smap['DC_PV_W']
    p_pv = p_pv_hist.iloc[-1]
    p_pv_hist = p_pv_hist.drop(index=p_pv_hist.index[-1]) # Delete latest observation
    p_pv_hist = p_pv_hist.resample('15T', base=p_pv_hist.index[-1].minute).mean().interpolate()
    p_pv_hist = p_pv_hist.iloc[-(60/15*24*2)::]
    p_pv_hist.index = (p_pv_hist.index.astype(np.int64) / 10 ** 6).astype(np.str)
    p_pv_hist = p_pv_hist.to_dict()
    return p_pv, p_pv_hist

def st_to_now(st):
    return (st - datetime(1970,1,1)).total_seconds()

In [ ]:
if run_component_tests:
    # get historic PV from SMAP (warm start)
    print ('Single Inverter')
    p_pv, p_pv_hist = warmstart_mpc_input(time.time(), inv_id=1)
    temp = pd.Series(p_pv_hist)
    temp.index = pd.to_datetime(temp.index, unit='ms')
    temp.plot()
    plt.show()
    print ('Average of all Inverter')
    p_pv, p_pv_hist = warmstart_mpc_input(time.time(), inv_id='sum')
    temp = pd.Series(p_pv_hist)
    temp.index = pd.to_datetime(temp.index, unit='ms')
    temp.plot()
    plt.show()

In [ ]:
if run_component_tests:
    import sys
    # Test controller
    dynamicforecast = mpc_input()
    # Get all variables
    print dynamicforecast.get_model_variables()

    now = time.time()
    tz = -8

    # get historic PV from SMAP (warm start)
    p_pv, p_pv_hist = warmstart_mpc_input(time.time())
    temp = pd.Series(p_pv_hist)
    temp.index = pd.to_datetime(temp.index, unit='ms')
    temp.plot()

    # Makeup some inputs
    inputs = {'time':time.time(),'p_pv':p_pv,'p_pv_hist_init':p_pv_hist, \
              'location_latitude':37.87, 'location_longitude':-122.27, 'tz_st':-8}

    models = {}
    models['regression'] = {'history':4, 'prediction':2}
    models['sarima'] = {'path':os.path.join(optimization_root,'Forecasting','models/all/20181031/SARIMA_model_20181025.json'), \
                        'normPowerCoeff': 2626.0}
    models['nn'] = {'path':os.path.join(optimization_root,'Forecasting','models/all/20181031/NNmodel_best_20181025.sav'), \
                    'normPowerCoeff': 2626.0, 'normInputData':True, \
                    'normTa': 30.0, 'normCC': 100.0, 'normCS': 1000.0, \
                    'architecture':'scalar', \
        #                       temp  cloud sky   Pd-1  hor                    
                    'inputData':[True, True, True, True, True],
                    'randomSeed': 10}
    models['alpha'] = {'path':os.path.join(optimization_root,'Forecasting','models/all/20181031/optimal_weighting_factors_20181031.json')}

    inputs['models'] = models
    #inputs['config'] = {'pv_norm':11682}
    inputs['config'] = {'scale_pv':1, 'scale_load':0.5}
    inputs['p_pv_hist'] = None
    #from MPCforecasting_V2 import example_inputs1
    #obsDf, wfDf, PVactual, model_paths, config = example_inputs1()
    #inputs['wf_prev'] = wfDf
    # Query controller
    print 'Log-message', dynamicforecast.do_step(inputs=inputs), '\n' # Display log message
    #print 'Output', dynamicforecast.get_output(keys=['load_demand']), '\n'

    keys = ['generation_pv','load_demand','p_pv_hist_update','forecast_pv_sarima','forecast_pv_nn']
    out_df = dynamicforecast.get_output(keys=keys)
    out_df = pd.DataFrame().from_dict(out_df)
    out_df.index = pd.to_datetime(out_df.index, unit='ms')
    print 'Historic PV:\n', out_df['p_pv_hist_update'][~np.isnan(out_df['p_pv_hist_update'])].iloc[-3::], '\n'
    print 'Forecast PV:\n', (out_df['generation_pv'][~np.isnan(out_df['generation_pv'])]*1e3).iloc[0:3], '\n'
    (out_df['generation_pv']*1e3).plot()
    out_df['forecast_pv_sarima'].plot()
    out_df['forecast_pv_nn'].plot()
    plt.legend(loc=2)
    plt.show()
    out_df['forecast_pv_nn'].plot()
    plt.show()

In [ ]:
if run_component_tests:
    import sys
    # Test controller
    dynamicforecast = mpc_input()
    # Get all variables
    #print dynamicforecast.get_model_variables()

    now = time.time()
    tz = -8

    # get historic PV from SMAP (warm start)
    p_pv, p_pv_hist = warmstart_mpc_input(time.time())
    temp = pd.Series(p_pv_hist)
    temp.index = pd.to_datetime(temp.index, unit='ms')
    #temp.plot()

    # Makeup some inputs
    inputs = {'time':time.time(),'p_pv':p_pv,'p_pv_hist_init':p_pv_hist, \
              'location_latitude':37.87, 'location_longitude':-122.27, 'tz_st':-8}

    models = {}
    models['regression'] = {'history':4, 'prediction':2}
    models['sarima'] = {'path':os.path.join(optimization_root,'Forecasting','models/all/20181031/SARIMA_model_20181025.json'), \
                        'normPowerCoeff': 2626.0}
    models['nn'] = {'path':os.path.join(optimization_root,'Forecasting','models/all/20181031/NNmodel_best_20181025.sav'), \
                    'normPowerCoeff': 2626.0, 'normInputData':True, \
                    'normTa': 30.0, 'normCC': 100.0, 'normCS': 1000.0, \
                    'architecture':'scalar', \
        #                       temp  cloud sky   Pd-1  hor                    
                    'inputData':[True, True, True, True, True],
                    'randomSeed': 10}
    models['alpha'] = {'path':os.path.join(optimization_root,'Forecasting','models/all/20181031/optimal_weighting_factors_20181031.json')}

    inputs['models'] = models
    #inputs['config'] = {'pv_norm':11682}
    inputs['config'] = {'scale_pv':1, 'scale_load':0.5}
    inputs['p_pv_hist'] = None
    inputs['forecast_pv_prev'] = 10
    inputs['forecast_load_prev'] = 10
    #from MPCforecasting_V2 import example_inputs1
    #obsDf, wfDf, PVactual, model_paths, config = example_inputs1()
    #inputs['wf_prev'] = wfDf
    # Query controller
    for i in range(3):
        

        if i > 0:
            inputs['wf_prev'] = dynamicforecast.output['weather_forecast']
            inputs['p_pv_hist'] = dynamicforecast.output['p_pv_hist_update']
            dynamicforecast.input['test_forcast'] = 1
        
        
        print 'Log-message', dynamicforecast.do_step(inputs=inputs), '\n' # Display log message
        #print 'Output', dynamicforecast.get_output(keys=['load_demand']), '\n'

        keys = ['generation_pv','load_demand','p_pv_hist_update','forecast_pv_sarima','forecast_pv_nn']
        out_df = dynamicforecast.get_output(keys=keys)
        out_df = pd.DataFrame().from_dict(out_df)
        out_df.index = pd.to_datetime(out_df.index, unit='ms')
        print 'Historic PV:\n', out_df['p_pv_hist_update'][~np.isnan(out_df['p_pv_hist_update'])].iloc[-3::], '\n'
        print 'Forecast PV:\n', (out_df['generation_pv'][~np.isnan(out_df['generation_pv'])]*1e3).iloc[0:3], '\n'
        (out_df['generation_pv']*1e3).plot()
        out_df['forecast_pv_sarima'].plot()
        out_df['forecast_pv_nn'].plot()
        plt.legend(loc=2)
        plt.show()

        time.sleep(5)